# Optimization with chance constraints

A model to calculate the optimal number of workers in an 8-hour shift. The workers change shifts either at the beginning of the shift or at the middle, so that at every given shift workers from the current shift and the previous shift are present. We know the mean value of the required number of workers, but this requirement is stochastic. We suppose that it follows a Poisson distribution. For example if a certain 4 hour window requires 9 workers, then

In [1]:
cat('95% of the time the required workers will be less than', qpois(0.95,9),
    'and more than', qpois(0.95,9,lower.tail = F))

95% of the time the required workers will be less than 14 and more than 4

Requirements:
   - Staff is required 24 hours a day
   - Employees work 8-hour shifts
   - Employee shift starts every 4 hours

Assume that we have the following mean rates and we set the constraint so that 60% of the time they will be satisfied, that is the **VaR** or **value at risk** of the problem.
The model:

Variables: $X_i$, $i=1:6$, the number of workers

Constraints:
AvailStaff $(X_i) >=$ VaR(RequiredStaff;ChanceCtr), $i=1:6$

Objective:
$\text{Minimise: } ( sum_i X_i)$

In [2]:
options(warn=-1)
mean_rates = c(9,22,25,17,30,13)
CHANCECTR = 0.6 #60% of the times the requirements will be satisfied. The higher the bigger the value.

In [3]:
library(data.table)
library(lpSolveAPI)
library(lubridate)
library(stringr)

DF = data.frame(val = 1:6,
                dt_start = paste0(as.character(seq(0,20,by=4)),':','00')  ,
                dt_end = paste0(as.character(seq(4,24,by=4)),':','00') ,
                effective_rate_ctr = sapply(mean_rates, function(x) qpois(CHANCECTR,x)),
                mean_rates = mean_rates,
                conf_95_lower = sapply(mean_rates, function(x) qpois(0.95,x)),
                conf_95_uper = sapply(mean_rates, function(x) qpois(0.95,x, lower.tail=F)),
                stringsAsFactors = FALSE)

DF


Attaching package: ‘lubridate’

The following objects are masked from ‘package:data.table’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year

The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union



val,dt_start,dt_end,effective_rate_ctr,mean_rates,conf_95_lower,conf_95_uper
1,0:00,4:00,10,9,14,4
2,4:00,8:00,23,22,30,15
3,8:00,12:00,26,25,33,17
4,12:00,16:00,18,17,24,11
5,16:00,20:00,31,30,39,21
6,20:00,24:00,14,13,19,7


In [4]:
#Initialize model and build objective
tVar = length(mean_rates)
lprec = make.lp(tVar,tVar)
#Build variables
write_objective = function(nVar){
  VarX_col_idx = 1:nVar
  obj = rep(1,nVar)
  return(list(VarX_col_idx,
              obj))
}

#At each 4-hour the workers of the current and previous shift are present
avail_staff = function(X,i,n){
  ifelse(i>1,X[i] + X[i-1],X[1] + X[n])
}


In [5]:

write_X_requirements = function(t_idx,tVar){
  
  row_idx = rep(0,tVar) #the coefficient of all the variables
  if(t_idx>1){
    row_idx[c(t_idx, t_idx-1)] = -1
  }else{
    row_idx[c(t_idx, tVar)] = -1  
  }
  return(row_idx)
  
}

write_constraints = function(model, mean_rates,tVar){
  
  all_rows = c()
  all_dirs = c()
  all_rhss = c()
  n_rows = 0
  
  #VaR contraints
  print("Writing value at risk constraints ...")
  for (t_idx in 1:tVar){
    var_risk = write_X_requirements(t_idx, tVar)
    all_rows = append(all_rows, var_risk)
    all_dirs = append(all_dirs, "<=")
    all_rhss = append(all_rhss, -qpois(CHANCECTR,mean_rates[t_idx]))
    n_rows = n_rows + 1
    set.row(model,n_rows,var_risk)
  }

f.con = matrix(all_rows, nrow = n_rows, byrow = TRUE)
f.dir = all_dirs
f.rhs = all_rhss
return(list(f.con, f.dir, f.rhs))

}


In [6]:

constraints = write_constraints(lprec,mean_rates,tVar)
f.con = constraints[[1]]
f.dir = constraints[[2]]
f.rhs = constraints[[3]]
f.obj = write_objective(tVar)[[2]]
f.x_idx = write_objective(tVar)[[1]]


[1] "Writing value at risk constraints ..."


In [7]:
#solve
set.constr.value(lprec,
                 lhs = rep(NA, tVar),
                 rhs = f.rhs,
                 1:tVar)

set.objfn(lprec, f.obj)
set.type(lprec,columns = f.x_idx, type = c("integer"))
solStatusCode =  solve.lpExtPtr(lprec)
sols = get.variables(lprec)
minimum = get.objective(lprec)

cat("Solution status: ",solStatusCode, "\n", "minimum: ", minimum, "\n", sols)


Solution status:  0 
 minimum:  67 
 10 25 1 17 14 0

Calculate the available staff (from the two shifts) and the chances that the day will require no more workers than the solution

In [8]:
available_staff = sapply(1:length(mean_rates),
                         function(x) avail_staff(sols, x, length(mean_rates)) )

chances = sapply(1:length(mean_rates), function(x) ppois(available_staff[x],
                                           mean_rates[x]))

DF$solution = sols
DF$available_staff= available_staff
DF$chances = chances


DF

val,dt_start,dt_end,effective_rate_ctr,mean_rates,conf_95_lower,conf_95_uper,solution,available_staff,chances
1,0:00,4:00,10,9,14,4,10,10,0.7059883
2,4:00,8:00,23,22,30,15,25,35,0.9962450
3,8:00,12:00,26,25,33,17,1,26,0.6293858
4,12:00,16:00,18,17,24,11,17,18,0.6549584
5,16:00,20:00,31,30,39,21,14,31,0.6186430
6,20:00,24:00,14,13,19,7,0,14,0.6751315
